In [ ]:
# Bibliotecas necessárias
import geopandas as gpd
import pandas as pd
import glob
import os
from pathlib import Path
import shutil

In [ ]:
def find_shape_files(directory_path):
    list_shape_files = []
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            if file.endswith('.shp'):
                file_path = Path(root) / file
                list_shape_files.append(file_path)
    return list_shape_files

def find_infos(row, directory_path):
    my_url = row['url']
    url_path = Path(my_url)
    relative_path = url_path.relative_to(directory_path)

    parts = list(relative_path.parts)
    if len(parts) >= 3:
        sub_1, sub_2, nome_shape = parts[-3:]
    else:
        sub_1, sub_2, nome_shape = '', '', parts[-1]
    return (
        str(url_path),
        sub_1,
        sub_2,
        nome_shape
    )

# Função para encontrar o tipo de arquivo
def find_file_type(name):
    parts = name.split('_', maxsplit=2)
    if len(parts) < 3:
        return "Unknown"
    return parts[2].split('.')[0]

# Função para obter o caminho
def get_path_f(folder):
    states = {}
    for state_dir in os.listdir(folder):
        if os.path.isdir(os.path.join(folder, state_dir)):
            state_name = state_dir
            state_path = os.path.join(folder, state_dir)
            states[state_name] = state_path
    return states

In [ ]:
from pathlib import Path
import pandas as pd
import geopandas as gpd

# Definindo os caminhos de entrada e saída como objetos Path
input_path = Path(r'D:\1_A_Projetos\BD\FBDS\RO\RO')
output_path = Path(r'D:\1_A_Projetos\BD\FBDS\RO\Ajuste')  # Certifique-se de que este caminho esteja correto

In [ ]:
print(output_path)

In [ ]:
# Funções fictícias para simular o comportamento do código original
def find_shape_files(caminho):
    # Retorna uma lista de arquivos .shp no diretório (função fictícia para exemplo)
    return list(caminho.rglob("*.shp"))

def find_infos(row, caminho):
    # Simulação para a criação de colunas (função fictícia para exemplo)
    return [str(caminho), "sub1", "sub2", "nome_shape"]

def find_file_type(nome_shape):
    # Simulação para identificar o tipo do arquivo (função fictícia para exemplo)
    return "tipo_shape"

# Gerar dicionário de estados dinamicamente percorrendo subpastas
states_root = {estado.name: estado for estado in input_path.iterdir() if estado.is_dir()}

# Processando cada estado
for estado, caminho in states_root.items():
    print(f"Estado: {estado}, Caminho: {caminho}")
    
    # Encontre todos os arquivos .shp no diretório e suas subpastas
    list_shape_files = find_shape_files(caminho)
    
    # Crie DataFrame
    df = pd.DataFrame(list_shape_files, columns=['url'])
    
    # Crie colunas
    df[['path', 'sub_1', 'sub_2', 'nome_shape']] = df.apply(
        lambda row: pd.Series(find_infos(row, caminho)),
        axis=1,
    )
    df['nome_tipo'] = df['nome_shape'].apply(find_file_type)
    
    # Definir o caminho como objeto Path
    caminhoW = Path(caminho)
    
    # Listas de arquivos APP, HIDROGRAFIA e USO
    list_files_app = list(caminhoW.rglob('*APP/*APP.shp'))
    list_files_app_uso = list(caminhoW.rglob('*APP/*APP_USO.shp'))
    
    list_files_hidro_simples = list(caminhoW.rglob('*HIDROGRAFIA/*RIOS_SIMPLES.shp'))
    list_files_hidro_duplas = list(caminhoW.rglob('*HIDROGRAFIA/*RIOS_DUPLOS*.shp'))
    list_files_hidro_nascentes = list(caminhoW.rglob('*HIDROGRAFIA/*NASCENTE*.shp'))
    list_files_hidro_massa = list(caminhoW.rglob('*HIDROGRAFIA/*MASSA*_DAGUA*.shp'))
    
    list_files_uso = list(caminhoW.rglob('*USO/*USO*.shp'))
    
    # Dicionário de listas de dados
    dict_lists = {
        'app': list_files_app,
        'app_uso': list_files_app_uso,
        'hidro_simples': list_files_hidro_simples,
        'hidro_duplas': list_files_hidro_duplas,
        'hidro_nascentes': list_files_hidro_nascentes,
        'hidro_massa': list_files_hidro_massa,
        'uso': list_files_uso
    }
    
    # Loop sobre os itens do dicionário de listas de dados
    for k, v in dict_lists.items():
        if v:  # Certifica-se de que a lista não está vazia
            # Etapa 2: Junta todos os arquivos shapefiles em um único GeoDataFrame
            list_shp = [gpd.read_file(shp).to_crs(epsg=4326) for shp in v]
            gdf = gpd.GeoDataFrame(pd.concat(list_shp, ignore_index=True), crs="EPSG:4326")
            
            # Etapa 3: Salvar o GeoDataFrame como um arquivo .gpkg
            gdf.to_file(
                output_path / f'{estado}_{k}.gpkg',  # Corrigido com o uso do Path
                layer=f'{k}',
                driver='GPKG',
                overwrite=True  # Use o argumento correto 'overwrite' se necessário
            )
            print(f'Fim do {k}')
        
    # Liberar variáveis após cada iteração
    del list_shape_files, df, caminhoW, list_files_app, list_files_app_uso
    del list_files_hidro_simples, list_files_hidro_duplas, list_files_hidro_nascentes, list_files_hidro_massa
    del list_files_uso, dict_lists, list_shp, gdf
    
    # Fim da iteração do estado
    print(f'Fim da iteração para {estado}')

In [ ]:
import geopandas as gpd
import glob
import os
import pandas as pd

In [ ]:
# Defina o diretório onde os arquivos .gpkg estão localizados
diretorio_origem = r"D:\1_A_Projetos\BD\FBDS\RO\Ajuste"

# Defina o diretório onde os arquivos combinados serão salvos
diretorio_destino = r"D:\1_A_Projetos\BD\FBDS\RO\Fusao"

In [ ]:
# Verifique se o diretório de destino existe; caso contrário, crie-o
if not os.path.exists(diretorio_destino):
    os.makedirs(diretorio_destino)

# Listas de padrões para os diferentes tipos de arquivos que você quer combinar
tipos = ['app', 'app_uso', 'hidro_simples', 'hidro_duplas', 'hidro_nascentes', 'hidro_massa', 'uso']

# Loop para cada tipo de arquivo
for tipo in tipos:
    # Encontrar todos os arquivos que contêm o tipo no nome
    arquivos = glob.glob(os.path.join(diretorio_origem, f'*{tipo}*.gpkg'))
    
    # Verificar se foram encontrados arquivos para o tipo
    if len(arquivos) == 0:
        print(f"Nenhum arquivo encontrado para o tipo '{tipo}'")
        continue  # Pula para o próximo tipo

    print(f"Arquivos encontrados para '{tipo}': {arquivos}")
    
    # Lista para armazenar os geodataframes
    gdfs = []
    
    # Ler e armazenar cada arquivo na lista
    for arquivo in arquivos:
        gdf = gpd.read_file(arquivo)
        gdfs.append(gdf)
    
    # Verificar se a lista de GeoDataFrames não está vazia antes de concatenar
    if len(gdfs) > 0:
        # Concatenar todos os GeoDataFrames em um único
        gdf_total = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))
        
        # Exportar para o diretório de destino
        gdf_total.to_file(os.path.join(diretorio_destino, f'RO_{tipo}.gpkg'), driver='GPKG')
        print(f"Arquivo RO_{tipo}.gpkg salvo com sucesso!")
    else:
        print(f"Nenhum GeoDataFrame válido encontrado para o tipo '{tipo}'.")

print("Processo concluído.")